# Deep End-to-end Causal Inference: Demo Notebook

This notebook provides a showcase of the features provided by our open source code for Deep End-to-end Causal Inference (DECI).

 - We begin with a simple two node example, showing how DECI can orient an edge correctly when non-Gaussian noise is present, and how DECI can then be used for treatment effect estimation
 - We show how different graph constraints can be incorporated into DECI
 - We showcase DECI on a larger graph example

## Notebook Setup

### Setup on AML
You can setup AML environment for causica by running the following commands on the terminal. Then, before running the notebook make sure you selected "causica" as your environment on top-right.

    conda create -n "causica" python==3.9 ipython
    conda activate causica
    conda install -y pip
    conda install -y ipykernel
    python -m ipykernel install --user --name causica --display-name causica
    sudo apt install graphviz-dev
    pip install git+https://github.com/microsoft/causica@9826ee9ba7dd63d72aaa0b3e0fd09636e38dd5bc
    pip install azureml-mlflow==1.46.0

<!-- pip install -r requirements.txt -->

### Setup on Databricks
Uncomment the following cell that will init-script on dbfs; set it up on your cluster. 
You can find more info on how to do so in this [link](https://learn.microsoft.com/en-us/azure/databricks/clusters/init-scripts#configure-a-cluster-scoped-init-script).

In [1]:
# %python
# dbutils.fs.put(
#     "dbfs:/databricks/init_scripts/install-pygraphviz-causica.sh",
#     """
#     #!/bin/bash
#     #install dependent packages
#     sudo apt-get install -y python3-dev graphviz libgraphviz-dev pkg-config
#     pip install pygraphviz
#     pip install git+https://github.com/microsoft/causica@9826ee9ba7dd63d72aaa0b3e0fd09636e38dd5bc""", True)

## Imports

In [2]:
import sys
sys.path.append("../..")

In [3]:
from causica_pkg.causica.models.deci.rhino import Rhino


In [4]:
# Some imports to get us started
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['PYTHONWARNINGS'] = 'ignore::FutureWarning'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Utilities
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Causica imports
# from causica_pkg.causica.models.deci.deci import DECI
from causica_pkg.causica.datasets.dataset import Dataset
from causica_pkg.causica.datasets.variables import Variables
from causica_pkg.causica.experiment.steps.step_func import load_data

# Plots
import seaborn as sns
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# device: change cuda to cpu if GPU is not available; we recommend gpu training
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

## Data Preparation
We read the data from BogoVirus with random dose policy. We then convert the data in a format containing information of prev. and cur. days. Patients are sampled into train-test files. All the records of the the patients in trainset is used to train DECI model. The test set is used for validation of ATE estimations, which is not covered in this notebook.

The dataset is simulated before and here we only load it from SQL:

In [5]:
import pandas as pd

df = pd.read_csv("../../data/bogo/cnt_100_patients.csv")
df.head()

,patient_id,cohort,day_number,infection,severity,drug,cum_drug,efficacy,outcome,reward
0,9018,10,1,30,18.672222,0.2,0.061564,2.254764,NaN,-1
1,9018,10,2,37,23.850061,0.5,0.186697,5.611975,NaN,-1
2,9018,10,3,40,24.831434,0.0,0.128113,0.154652,NaN,-1
3,9018,10,4,46,31.784742,0.5,0.238457,5.773507,NaN,-1
4,9018,10,5,52,33.240071,0.5,0.313421,5.461095,NaN,-1


In [6]:
df["outcome"]=df["outcome"].fillna("stay")

In [7]:
df_rhino = df.sort_values(by=["patient_id", "day_number"]).loc[:,
    ["patient_id", "infection", "severity", "drug", "cum_drug", "efficacy", "outcome"]
]

df_rhino.head()

,patient_id,infection,severity,drug,cum_drug,efficacy,outcome
89,495,37,18.332330,0.1,0.028877,0.618856,stay
90,495,41,23.319683,0.2,0.080129,2.530860,stay
91,495,44,27.401423,0.4,0.176783,4.378567,stay
92,495,44,29.735350,0.5,0.277358,6.129380,stay
93,495,53,30.711535,0.5,0.339562,5.858048,stay


In [8]:
df_rhino.to_csv("../../data/bogo/all.csv", index=False)

In [9]:
dataset_config = {
    "dataset_format": "temporal_causal_csv",
    "timeseries_column_index": 0,
    "use_predefined_dataset": False,
    "random_seed": [0],
    "negative_sample": False
}

# DECI configurations
model_hyperparams={
  "ICGNN_embedding_size": None,
  "additional_spline_flow": 0,
  "allow_instantaneous": False,
  "base_distribution_type": "gaussian",
  "cate_rff_lengthscale": [
    0.1,
    1
  ],
  "cate_rff_n_features": 3000,
  "conditional_decoder_layer_sizes": [
    10
  ],
  "conditional_embedding_size": 16,
  "conditional_encoder_layer_sizes": [
    10
  ],
  "conditional_spline_order": "linear",
  "decoder_layer_sizes": [
    10
  ],
  "encoder_layer_sizes": [
    10
  ],
  "imputation": False,
  "init_logits": [
    -3,
    -3
  ],
  "lag": 2,
  "lambda_dag": 10,
  "lambda_prior": 100000,
  "lambda_sparse": 15,
  "norm_layers": True,
  "prior_A_confidence": 0.5,
  "random_seed": 0,
  "res_connection": True,
  "spline_bins": 8,
  "tau_gumbel": 0.25,
  "var_dist_A_mode": "temporal_three"
}
training_hyperparams={
    "learning_rate": 0.001,
    "likelihoods_learning_rate": 0.001,
    "batch_size": 64,
    "stardardize_data_mean": False,
    "stardardize_data_std": False,
    "rho": 1.0,
    "safety_rho": 10000000000000.0,
    "alpha": 0.0,
    "safety_alpha": 10000000000000.0,
    "tol_dag": -1,
    "progress_rate": 0.65,
    "max_steps_auglag": 60,
    "max_auglag_inner_epochs": 6000,
    "max_p_train_dropout": 0,
    "reconstruction_loss_factor": 1.0,
    "anneal_entropy": "noanneal"
}

### Create variables & Dataset objects

In [10]:
# Method for creating variables.json
def create_variables(df, categorical_vars={}, timeseries_column_index=0, variables_json_path=None, text_vars=None, target_column_names=[]):
    if text_vars is None:
        text_vars = []
    variables_info = []

    for ident, column_name in enumerate(df.columns):
        # if ident == timeseries_column_index:
        # continue
        query = column_name if target_column_names is not None else ident != len(df.columns) - 1
        var = {
            "id": ident,
            "query": query,
            "name": column_name,
        }
        if column_name not in text_vars:
            var["type"] = "categorical" if column_name in categorical_vars else "continuous"
            var["lower"] = 0 if column_name in categorical_vars else np.nanmin(df[column_name])
            var["upper"] = (
                categorical_vars[column_name] - 1 if column_name in categorical_vars else np.nanmax(df[column_name])
            )
        elif column_name in text_vars:
            var["type"] = "text"
            var["overwrite_processed_dim"] = 768  # Sentence Transformer model has that dimension

        if column_name in target_column_names:
            var["group_name"]="targets"
        variables_info.append(var)

    variable_dict = {"variables": variables_info, "metadata_variables": []}
    del variable_dict["variables"][timeseries_column_index]
    variables = Variables.create_from_dict(variable_dict)
    if variables_json_path:
        variables.save(variables_json_path)
    return variables

In [11]:
from causica_pkg.causica.experiment.run_context import RunContext
run_context = RunContext()

In [12]:
from causica_pkg.causica.datasets.dataset import Dataset
from causica_pkg.causica.datasets.variables import Variables

# Set up data in a suitable form for DECI to consume, using the loaded data types
mask_data = ~df_rhino.isna()
data_mask = mask_data.to_numpy().astype(int) # np.ones(numpy_data.shape)

# na to zero
df_rhino=df_rhino.fillna(0)
numpy_data = df_rhino.to_numpy()

In [19]:
numpy_data

array([[495, 37, 18.332329910196396, ..., 0.0288769015219383,
        0.6188559705515274, 'stay'],
       [495, 41, 23.319682971669756, ..., 0.0801294139741463,
        2.5308602568804144, 'stay'],
       [495, 44, 27.40142312571836, ..., 0.1767826080763779,
        4.378566919844883, 'stay'],
       ...,
       [98807, 83, 88.53074293036846, ..., 0.6942986198902211,
        4.65661614782621, 'stay'],
       [98807, 84, 99.89051714011596, ..., 0.8490488597814572,
        0.6065761843540785, 'stay'],
       [98807, 91, 118.86438978782888, ..., 1.3481529949986648,
        0.117537098830667, 'die']], dtype=object)

In [13]:
# no missingness
pd.isna(numpy_data).sum()

0

In [14]:
data_mask.sum()

9695

In [15]:
data_mask.shape[0] * data_mask.shape[1]

9695

In [16]:
variables = create_variables(df, categorical_vars={"outcome":3}, timeseries_column_index=0)
dataset = Dataset(train_data=numpy_data, train_mask=np.ones(numpy_data.shape), variables=variables)

In [17]:
# we create a directory to save the results of all the experiments
outputs_root = "../../runs/"
os.makedirs(outputs_root, exist_ok=True)

model_id = f"bogo_{datetime.now():%Y_%m_%d_%H%M%S}"
model = Rhino.create(model_id, save_dir=os.path.join(outputs_root, model_id), variables=variables, model_config_dict=model_hyperparams, device=device)
# model.set_graph_constraint(constraint_matrix)
model.run_train(dataset, training_hyperparams)

AssertionError: 

In [ ]:
# def create_variables_json(df, categorical_vars, variables_json_path, text_vars=None, target_column_names=None):
#     """
#     The method returns Variables object (causica.datasets.variables.Variables), and also saves the json file; 
#     The saved json file is for potential future use and the returned object is sufficient for what causica expects. 

#     df <pandas.DataFrame> dataset that variable types and boundries will be extracted from
#     categorical_vars <dict> a dictionary of categorical variables & the number of categories for each. e.g., {"gender":2, "race": 6}.
#     variables_json_path <str> filepath to save the variables dictionary

#     returns <causica.datasets.variables.Variables> variables
#     """
#     if text_vars is None:
#         text_vars = []
#     variables_info = []

#     for ident, column_name in enumerate(df.columns):
#         query = True if target_column_names is None else column_name not in target_column_names 
#         var = {
#             "id": ident,
#             "query": query,
#             "name": column_name,
#         }
#         if column_name not in text_vars:
#             var["type"] = "categorical" if column_name in categorical_vars else "continuous"
#             var["type"] = "binary" if (var["type"]=="categorical" and categorical_vars[column_name]==2) else var["type"]
#             var["lower"] = 0 if column_name in categorical_vars else np.nanmin(df[column_name])
#             var["upper"] = (
#                 categorical_vars[column_name] - 1 if column_name in categorical_vars else np.nanmax(df[column_name])
#             )
#         elif column_name in text_vars:
#             var["type"] = "text"
#             var["overwrite_processed_dim"] = 768  # Sentence Transformer model has that dimension

#         if target_column_names and column_name in target_column_names:
#             var["group_name"]="targets"
#         variables_info.append(var)
#     variables = Variables.create_from_dict({"variables": variables_info, "metadata_variables": []})
#     variables.save(variables_json_path)
#     return variables

In [ ]:
# # This data pre-processing treat all variables as continuous, although some of them are ordinal --  categorical_vars={'status':3}
# variables_json_path=data_dir + 'bogo_time/variables.json'
# os.makedirs(data_dir + 'bogo_time', exist_ok=True)

# # create variables object
# variables = create_variables_json(df_all, categorical_vars={'die':2, 'recover':2,'prev1_die':2, 'prev1_recover':2}, variables_json_path=variables_json_path, text_vars=None, target_column_names=[])
# varindex_to_name = {i: var.name for (i, var) in enumerate(variables)}
# varname_to_index = {var.name:i for (i, var) in enumerate(variables)}

### Create Dataset object

In [ ]:
# def create_dataset_obj(df_input, variables):
#   # Set up data in a suitable form for DECI to consume, using the loaded data types
#   # data_mask = ~df_input.isna().to_numpy() # np.ones(numpy_data.shape)
#   data_mask = ~np.isnan(df_input.to_numpy()) # deci can tolerate missing data; this mask help DECI distinguish 

#   # na to zero
#   numpy_data=df_input.fillna(0).to_numpy()
#   # print(numpy_data)
#   dataset = Dataset(train_data=numpy_data, train_mask=data_mask, variables=variables)
#   return dataset

# # create dataset object for train-data
# train_dataset = create_dataset_obj(df_train_cohort, variables)

In [ ]:
# df_train_cohort.head()

# Part 1: Graph Discovery

The following snippets step through the process of:
 - configuring and creating a DECI model
 - training a model with no graph constraints
 - training a model with graph constraints
 
*Note*: if you have already trained a DECI model, you can reload the model by skipping to the section below on "Loading a saved DECI model"

In [ ]:
# # we create a directory to save the results of all the experiments
# outputs_root = "bogovirus/runs/"
# os.makedirs(outputs_root, exist_ok=True)

In [ ]:
# device = "0" if torch.cuda.is_available() else "cpu"

## 1.1. Train with No Constraint

#### **Note** -- DECI configuration

The DECI model has a number of hyperparameters, but attention need not be paid to all of them. Here we highlight key hyperparameters that might be changed to improve performance:
 - `learning_rate` is the step size for the Adam optimizer. Decrease the learning rate if training is unstable
 - `var_dist_A_learning_rate` is the learning rate for the causal graph distribution
 - `standardize_data_mean` and `standardize_data_std` tell DECI to standarize the data by subtracting the mean and dividing by the standard deviation of each column. *Note*: this improves training, but you need to take care because treatment effects will now be computed in this standardized space
 - `max_auglag_inner_epochs` is the number of gradient steps taken in each "inner step" of the DECI optimization. Decrease this to speed up training (at the risk of training not converging). Increase this to learn a more accurate model, or when the dataset is larger.
 - `max_steps_auglag` is the maximum number of "inner steps" taken. Decrease this to end training early (e.g. before a DAG has been found) to speed up training
 - `base_distribution_type` is the type of DECI model that is trained. It should be either `gaussian` or `spline`. Use `spline` for highly non-Gaussian data, or to fit a better density model of the observational data.
 - `lambda_sparse` controls the coefficient of graph sparsity regularizer.
 
Other hyperparameters are less frequently changed.


To speed up training you can try the followings:
 - increasing `learning_rate`
 - increasing `batch_size` (reduces noise when using higher learning rate)
 - decreasing `max_steps_auglag` (go as low as you can and still get a DAG)
 - decreasing `max_auglag_inner_epochs`

Note that an undertrained model might be unsuccessful to converge to a DAG or may have less accurate predictions.

In [ ]:
# # DECI configurations
# model_config = {
#     "tau_gumbel": 0.25,
#     "lambda_dag": 200.0,
#     "lambda_sparse": 5.0,
#     "lambda_prior": 0.0,
#     # Choosing a Gaussian DECI model, alternative is "spline"/"gaussian"
#     "base_distribution_type": "spline",
#     "imputation": False,
#     "spline_bins": 8,
#     "var_dist_A_mode": "enco",
#     "mode_adjacency": "learn",
#     # ?
#     'norm_layers': True, 
#     'res_connection': True,
# }

# training_params = {
#     # Setting higher learning rates can speed up training, but risks some instability
#     "learning_rate": 1e-3,
#     "var_dist_A_learning_rate": 1e-2,
#     "batch_size": 512, #256,
#     # This standarizes the data before training. The DECI model operates in standardized space.
#     "standardize_data_mean": False, #True,
#     "standardize_data_std": False, #True,
#     "rho": 1.0,
#     "safety_rho": 1e18,
#     "alpha": 0.0,
#     "safety_alpha": 1e18,
#     "tol_dag": 1e-9,
#     "progress_rate": 0.65,
#     # We are setting this large to wait until we find a DAG
#     "max_steps_auglag": 50,
#     # We are setting this large to learn a more accurate model.
#     "max_auglag_inner_epochs": 2000,
#     "max_p_train_dropout": 0.2,
#     "reconstruction_loss_factor": 1.0,
#     "anneal_entropy": "noanneal",
# }

In [ ]:
df_train_cohort.head(2)

In [ ]:
# model_id = f"bogo_{datetime.now():%Y_%m_%d_%H%M%S}"
# model = DECI.create(model_id, save_dir=os.path.join(outputs_root, model_id), variables=variables, model_config_dict=model_config, device=device)
# # model.set_graph_constraint(constraint_matrix)
# model.run_train(train_dataset, training_params)

In [ ]:
# print("Saved as ", model_id)

In [ ]:
# model_id="bogo_2022_11_03_013822"
# # model = DECI.load(model_id, os.path.join(outputs_root, model_id), device=device)

In [ ]:
# pred_graph = model.networkx_graph()
# pred_graph = nx.relabel_nodes(pred_graph, varindex_to_name)

In [ ]:
# def topological_plot(G):
#     # G is a nx.DiGraph object
#     for layer, nodes in enumerate(nx.topological_generations(G)):
#         # `multipartite_layout` expects the layer as a node attribute, so add the
#         # numeric layer value as a node attribute
#         for node in nodes:
#             G.nodes[node]["layer"] = layer
#     # Compute the multipartite_layout using the "layer" node attribute
#     # pos = nx.multipartite_layout(G, subset_key="layer")
#     pos = nx.shell_layout(G) #, subset_key="layer")
#     fig, ax = plt.subplots()
#     nx.draw_networkx(G, pos=pos, ax=ax)
#     ax.set_title("DAG layout in topological order")
#     fig.tight_layout()
#     plt.show()

In [ ]:
# topological_plot(pred_graph)

## 1.2. Adding domain-specific graph constraints
To improve the quality of the learned graph, it is possible to place constraints on the graph that DECI learns. The constraints come in two flavours:
 - *negative constraints* mean a certain edge cannot exist in the graph,
 - *positive constraints* mean a certain edge must exist in the graph.

In [ ]:
# # Constraint matrix coding is described in the DECI docstrings
# # The constraint matrix has the same shape as the adjacency matrix
# # A `nan` value means no constraint
# # A 0 value means a negative constraint
# # A 1 value means a positive constraint

# constraint_matrix = np.full((len(varname_to_index.keys()), len(varname_to_index.keys())), np.nan, dtype=np.float32)

Temporal: First, we add temporal constraints, future does not cause the past

In [ ]:
# # temporal constraints
# prev_idxes = [varname_to_index[var] for var in varname_to_index.keys() if var.startswith("prev1")]
# cur_idxes = [varname_to_index[var] for var in varname_to_index.keys() if not var.startswith("prev1")]

# for prev_idx in prev_idxes:
#     constraint_matrix[cur_idxes, prev_idx] = 0.

Secondly, constraints on the outcomes not to be a cause for others

In [ ]:
# outcome_factors = ['die', 'recover']
# outcome_idxes = [varname_to_index[var] for var in outcome_factors]
# nonoutcome_idxes = [varname_to_index[var] for var in varname_to_index.keys() if var not in outcome_factors]

# for out_idx in outcome_idxes:
#     constraint_matrix[out_idx, nonoutcome_idxes] = 0.

In [ ]:
# constraint_matrix

In [ ]:
# model_id_c = model_id + "_c"
# model_c = DECI.create(model_id_c, save_dir=os.path.join(outputs_root, model_id_c), variables=variables, model_config_dict=model_config, device=device)
# model_c.set_graph_constraint(constraint_matrix)
# model_c.run_train(train_dataset, training_params)

In [ ]:
# pred_graph_c = model_c.networkx_graph()
# pred_graph_c = nx.relabel_nodes(pred_graph_c, varindex_to_name)
# topological_plot(pred_graph_c)

Thirdly, we know severity is a cause for sevirity_next, infection_prev to infection_next, and drug to cum_drug

In [ ]:
# constraint_matrix_c2 = constraint_matrix.copy()

# constraint_matrix_c2[varname_to_index['severity'], varname_to_index['drug']]=1.0
# constraint_matrix_c2[varname_to_index['prev1_severity'], varname_to_index['prev1_drug']]=1.0
# constraint_matrix_c2[varname_to_index['drug'], varname_to_index['severity']]=0.0
# constraint_matrix_c2[varname_to_index['prev1_drug'], varname_to_index['prev1_severity']]=0.0

# constraint_matrix_c2[varname_to_index['prev1_severity'], varname_to_index['severity']]=1.0
# constraint_matrix_c2[varname_to_index['prev1_infection'], varname_to_index['infection']]=1.0

# constraint_matrix_c2[varname_to_index['prev1_drug'], varname_to_index['severity']]=1.0
# constraint_matrix_c2[varname_to_index['prev1_cum_drug'], varname_to_index['severity']]=1.0

# constraint_matrix_c2[varname_to_index['drug'], varname_to_index['cum_drug']]=1.0
# constraint_matrix_c2[varname_to_index['cum_drug'], varname_to_index['drug']]=0.0

# constraint_matrix_c2[varname_to_index['prev1_drug'], varname_to_index['prev1_cum_drug']]=1.0
# constraint_matrix_c2[varname_to_index['prev1_cum_drug'], varname_to_index['prev1_drug']]=0.0

# constraint_matrix_c2

In [ ]:
# model_id_c2 = model_id + "_c2"
# model_c2 = DECI.create(model_id_c2, save_dir=os.path.join(outputs_root, model_id_c2), variables=variables, model_config_dict=model_config, device=device)
# model_c2.set_graph_constraint(constraint_matrix_c2)
# model_c2.run_train(train_dataset, training_params)

In [ ]:
# pred_graph_c2 = model_c2.networkx_graph()
# pred_graph_c2 = nx.relabel_nodes(pred_graph_c2, varindex_to_name)
# topological_plot(pred_graph_c2)

# Part 2: SimSim -- simulation for scenario analysis
DECI has also fitted an SCM that captures the functional relationship and error distribution of this dataset. We can estimate ATE and CATE using do-based sampling E\[X_{today}|do(X_{yesterday},drug=dose)].

## Treatment effect estimation -- Definitions

Treatment effects describe how a target variable (such as outcome_die) changes in response to an intervention on some other variables.
 - **Average Treatment Effect (ATE)** gives the difference between giving the drug to every patient, and not giving the treatment to any of them. This summarizes the overall or average effectiveness of the drug.
 - **Conditional Average Treatment Effect (CATE)** gives the difference between giving and not giving the drug, *restricted to a certain conditions or subgroup of patients*. For example, we could investigate the average treatment effect effect of drug on sever or nonsever patients (e.g., severity_now < 30. vs. severity_now > 109),.
 - **Individual Treatment Effect (ITE)** is the different in outcome_die between giving and not giving the drug *for one specific patient*.

In [ ]:
# import random
# from scipy import stats

# def is_out(row):
#     return (row[
#         "die"] > 0.9 or row[
#         "recover"] > 0.9 or row[
#         "prev1_die"] > 0.9 or row[
#         "prev1_recover"] > 0.9)
#     return False
    
# def get_drug(row, dose):
#     # depends on: severity
#     # treatment threshold for today
#     tx_threshold = random.randint(10, 50)
#     drug = dose if row['severity'] > tx_threshold else 0
#     return drug

# def get_next_day(model, prev_day, colnames, varname_to_index, drug_dose=None, sample_count=1):
#     cat_vars=[5, 6, 12, 13] #die/recover

#     intervention_idxs=[]
#     intervention_values=[]

#     vars = [col for col in colnames if col.startswith('prev1_')]
    
#     intervention_idxs += [varname_to_index[var] for var in vars]
#     intervention_values += [prev_day[var[len("prev1_"):]] for var in vars]

#     ### Model-based ATE estimate
#     next_day = model.sample(
#         sample_count, intervention_idxs=np.array(intervention_idxs), 
#         intervention_values=np.array(intervention_values)
#     ).cpu().numpy()
#     next_day_agg = next_day.mean(axis=0)

#     tx_threshold = random.randint(10, 50)
#     if (drug_dose is not None) and (next_day_agg[0] > tx_threshold): #severity
#         # drug dose
#         intervention_idxs = [varname_to_index[
#             "severity"], varname_to_index["drug"]] + intervention_idxs
#         intervention_values = [next_day_agg[0], drug_dose] + intervention_values

#         next_day = model.sample(
#                 sample_count, intervention_idxs=np.array(intervention_idxs), 
#                 intervention_values=np.array(intervention_values)
#             ).cpu().numpy()

#     next_day_agg = next_day.mean(axis=0)
#     next_day_agg[cat_vars] = stats.mode(next_day[:,cat_vars], axis=0).mode[0]
#     return next_day_agg # next_day.mean(axis=0)

# def run_simsim(model, population, df_core, df_prev, varname_to_index, num_of_days, dose):
#     # init
#     df_none=df_core.copy()
#     end_flags = [False] * population # dead or recovered

#     # foreach day
#     for k in range(1, num_of_days+1):
#         np_simk=np.empty_like(df_prev)
#         np_simk[:]=np.NaN

#         # foreach patient
#         for row_idx in range(np_simk.shape[0]):
#             row = df_prev.iloc[row_idx,:]
#             if end_flags[row_idx]:
#                 continue
#             elif is_out(row):
#                 # dead or recovered
#                 # print('P', str(row_idx), 'gone')
#                 end_flags[row_idx] = True
#                 continue

#             next_day = get_next_day(model, row, colnames=df_prev.columns, varname_to_index=varname_to_index, drug_dose=dose, sample_count=50)
#             # print(next_day)
#             np_simk[row_idx,:]=next_day
        
#         df_prev=pd.DataFrame(np_simk, columns=df_prev.columns)
#         # add day_number & patient_id
#         df_dayk=df_prev.copy()
#         # df_dayk.loc[:,df_prev.columns]=df_prev.loc[:,df_prev.columns].copy()
#         df_dayk["day_number"]= [k] * population
#         df_dayk["patient_id"]= df_core["patient_id"].values

#         # add to the db
#         df_none=df_none.append(df_dayk[df_none.columns], ignore_index=True)
#         alive = len(end_flags) - sum(end_flags)

#         if alive == 0:
#             break
#         print("Day", k, "is done, alive: ", alive)
#         if k%10 == 0:
#             df_none.to_csv(os.path.join(experiment_dir_c, 'simsim_data',str(k)+'.csv'))# No drug
#     return df_none

In [ ]:
# experiment_dir_c=os.path.join(outputs_root, model_id_c)
# os.mkdir(os.path.join(experiment_dir_c, 'simsim_data'))

In [ ]:
# experiment_dir_c2=os.path.join(outputs_root, model_id_c2)
# os.mkdir(os.path.join(experiment_dir_c2, 'simsim_data'))

Condition on a particular dose -- here dose=0.2; we do the same for all the dose candidate to find the optimal dose policy:

In [ ]:
# population=5000 #everyone
# df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() 

# #  setup
# df_prev = df_core[df_train_cohort.columns] # sample instead
# num_of_days=60
# dose = 0.2

# df02_c=run_simsim(model_c, population, df_core, df_prev, varname_to_index, num_of_days, dose)
# df02_c.to_csv(os.path.join(experiment_dir_c, 'simsim_data', 'deci_constrainted_dose0.2_max60days' +'.csv'))

# print(df02_c['die'].sum())
# print(df02_c['recover'].sum())


In [ ]:
# population=5000 #everyone
# df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() 

# #  setup
# df_prev = df_core[df_train_cohort.columns] # sample instead
# num_of_days=60
# dose = 0.2

# df02_c2=run_simsim(model_c2, population, df_core, df_prev, varname_to_index, num_of_days, dose)
# df02_c2.to_csv(os.path.join(experiment_dir_c2, 'simsim_data', 'deci_constrainted_dose0.2_max60days' +'.csv'))

# print(df02_c2['die'].sum())
# print(df02_c2['recover'].sum())
